In [1]:
import numpy as np
import math
import cv2

In [167]:
def bilinear_interpolation_ref(img, new_row_sz, new_col_sz):
    old_row_sz, old_col_sz = img.shape[:2]
    new_img = np.zeros(shape=(new_row_sz, new_col_sz), dtype=np.float64)
    
    row_scale = (old_row_sz-1) / (new_row_sz-1)
    col_scale = (old_col_sz-1) / (new_col_sz-1)
#     print("scales", row_scale, col_scale)
    
    row_mapping = []
    col_mapping = []
    
    row_nearest_idxs = []
    col_nearest_idxs = []
    
    row_deltas = []
    col_deltas = []
    new_col_idx = 1
    for new_row_idx in range(0, new_row_sz):
        for new_col_idx in range(0, new_col_sz):
            row_mapped_pos = row_scale * new_row_idx 
            row_mapping.append(row_mapped_pos)
            col_mapped_pos = col_scale * new_col_idx
            col_mapping.append(col_mapped_pos)
            
            row_nearest_idx = int(np.floor(row_mapped_pos))
#             row_nearest_idx = min(row_nearest_idx, old_row_sz - 2)
#             row_nearest_idx = max(row_nearest_idx, 1)
            
            row_nearest_idxs.append(row_nearest_idx)
            col_nearest_idx = int(np.floor(col_mapped_pos))
#             col_nearest_idx = min(col_nearest_idx, old_col_sz - 2)
#             col_nearest_idx = max(col_nearest_idx, 1)
            col_nearest_idxs.append(col_nearest_idx)
            
#             print("mapped postions", row_mapped_pos, col_mapped_pos, "-->", row_nearest_idx, col_nearest_idx)

            row_delta = row_mapped_pos - row_nearest_idx
            row_deltas.append(row_delta)
            col_delta = col_mapped_pos - col_nearest_idx
            col_deltas.append(col_delta)
#             print(new_row_idx, new_col_idx, "-->", row_nearest_idx, col_nearest_idx, "with deltas:", row_delta, col_delta)
            
            right_top_pixel = img[row_nearest_idx, col_nearest_idx] 
            
#             if row_nearest_idx == old_row_sz - 1:
#                 right_bottom_pixel = 0
#                 left_bottom_pixel = 0
#             elif col_nearest_idx == old_col_sz - 1:
#                 left_top_pixel = 0
#                 left_bottom_pixel = 0
#             else:
#                 right_bottom_pixel = img[row_nearest_idx+1, col_nearest_idx] 
#                 left_bottom_pixel = img[row_nearest_idx+1, col_nearest_idx+1]
#                 left_top_pixel = img[row_nearest_idx, col_nearest_idx+1] 
#                 left_bottom_pixel = img[row_nearest_idx+1, col_nearest_idx+1]
            
            y_idx = min(row_nearest_idx+1, old_row_sz-1)
            x_idx = min(col_nearest_idx+1, old_col_sz-1)
            right_bottom_pixel = img[y_idx, col_nearest_idx] 
            left_top_pixel = img[row_nearest_idx, x_idx] 
            left_bottom_pixel = img[y_idx, x_idx]
            
            new_pixel = right_top_pixel * (1 - row_delta)  * (1 - col_delta) + \
                        right_bottom_pixel  * (row_delta)  * (1 - col_delta) + \
                        left_top_pixel * (1 - row_delta)  * (col_delta) + \
                        left_bottom_pixel * (row_delta)  * (col_delta) 
            new_img[new_row_idx, new_col_idx] = new_pixel
    return new_img
    

In [ ]:
img = cv2.imread("./lena.png")
img = img[:7, :7, 0]; img

In [ ]:
%%timeit
bilinear_along_one_axis(img=img, new_row_sz=12, new_col_sz=12)

In [ ]:
cv_img = cv2.resize(img, (12, 12), interpolation = cv2.INTER_LINEAR); cv_img

In [ ]:
np.round(tmp).astype(np.uint8)

In [ ]:
np.round(tmp).astype(np.uint8) == cv_img

In [92]:
def do_bilinear_resize(img, new_size):
    '''
    Takes in a image with three channels and
    resizes to @param: new_size
    '''
    num_row_img, num_col_img, num_channels = img.shape
    resized_img = np.zeros((new_size[0], new_size[1], img.shape[-1]), dtype=np.uint8)
    num_row_resized_img, num_col_resized_img = resized_img.shape[:2]
    
    def get_nearest_idxs_with_deltas(old_sz, new_sz):
        scale = (old_sz - 1) / (new_sz -1)
        nearest_idxs, deltas = [], []
        for new_idx in range(0, new_sz):
            mapped_position = scale * new_idx
            nearest_idx = math.floor(mapped_position)
            delta = mapped_position - nearest_idx
            nearest_idxs.append(nearest_idx)
            deltas.append(delta)
            
#             print("idx mapped: ", new_idx, "-->", nearest_idx, "delta:", delta)
        return nearest_idxs, deltas
    
#     print("rows")
    row_nearest_idxs, row_deltas = get_nearest_idxs_with_deltas(old_sz=num_row_img, new_sz=num_row_resized_img)
#     print("cols")
    col_nearest_idxs, col_deltas = get_nearest_idxs_with_deltas(old_sz=num_col_img, new_sz=num_col_resized_img)
    
    for channel_idx in range(0, num_channels):
        for row_idx in range(0, num_row_resized_img):
            for col_idx in range(0, num_col_resized_img):
                row_nearest_idx = row_nearest_idxs[row_idx]
                col_nearest_idx = col_nearest_idxs[col_idx]
                row_delta = row_deltas[row_idx]
                col_delta = col_deltas[col_idx]
                
                row_idx_next = row_nearest_idx + 1
                row_idx_next = min(row_idx_next, num_row_img - 1)
                col_idx_next = col_nearest_idx + 1
                col_idx_next = min(col_idx_next, num_col_img - 1)

                right_top_pixel = img[row_nearest_idx, col_nearest_idx, channel_idx].astype(np.float32) 
                right_bottom_pixel = img[row_idx_next, col_nearest_idx, channel_idx].astype(np.float32) 
                left_top_pixel = img[row_nearest_idx, col_idx_next, channel_idx].astype(np.float32) 
                left_bottom_pixel = img[row_idx_next, col_idx_next, channel_idx].astype(np.float32)
                
                
                new_pixel = (right_top_pixel * (1 - row_delta)  * (1 - col_delta)).astype(np.float32) + \
                            (right_bottom_pixel  * (row_delta)  * (1 - col_delta)).astype(np.float32) + \
                            (left_top_pixel * (1 - row_delta)  * (col_delta)).astype(np.float32) + \
                            (left_bottom_pixel * (row_delta)  * (col_delta)).astype(np.float32)
#                 import pdb; pdb.set_trace()
                resized_img[row_idx, col_idx, channel_idx] = new_pixel
    return resized_img

In [189]:
img = cv2.imread("./lena.png")
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img.shape

(512, 512, 3)

In [101]:
res = do_bilinear_resize(img=img, new_size=(512//2, 512//2));
cv_res = cv2.resize(img, (256, 256), interpolation = cv2.INTER_LINEAR)
cv2.imwrite("lena_res_256.jpg", res)
cv2.imwrite("lena_res_cv_256.jpg", res)

True

In [102]:
res = do_bilinear_resize(img=img, new_size=(1024, 1024));
cv_res = cv2.resize(img, (1024, 1024), interpolation = cv2.INTER_LINEAR)
cv2.imwrite("lena_res_1024.jpg", res)
cv2.imwrite("lena_res_cv_1024.jpg", res)

False

In [ ]:
%%timeit
do_bilinear_resize(img, new_size=(12, 12));

In [83]:
res = do_bilinear_resize(img, new_size=(12, 12))[:,:,0]

rows
idx mapped:  0 --> 0 delta: 0.0
idx mapped:  1 --> 0 delta: 0.5454545454545454
idx mapped:  2 --> 1 delta: 0.09090909090909083
idx mapped:  3 --> 1 delta: 0.6363636363636362
idx mapped:  4 --> 2 delta: 0.18181818181818166
idx mapped:  5 --> 2 delta: 0.7272727272727271
idx mapped:  6 --> 3 delta: 0.2727272727272725
idx mapped:  7 --> 3 delta: 0.8181818181818179
idx mapped:  8 --> 4 delta: 0.3636363636363633
idx mapped:  9 --> 4 delta: 0.9090909090909083
idx mapped:  10 --> 5 delta: 0.45454545454545414
idx mapped:  11 --> 6 delta: 0.0
cols
idx mapped:  0 --> 0 delta: 0.0
idx mapped:  1 --> 0 delta: 0.5454545454545454
idx mapped:  2 --> 1 delta: 0.09090909090909083
idx mapped:  3 --> 1 delta: 0.6363636363636362
idx mapped:  4 --> 2 delta: 0.18181818181818166
idx mapped:  5 --> 2 delta: 0.7272727272727271
idx mapped:  6 --> 3 delta: 0.2727272727272725
idx mapped:  7 --> 3 delta: 0.8181818181818179
idx mapped:  8 --> 4 delta: 0.3636363636363633
idx mapped:  9 --> 4 delta: 0.90909090909

In [84]:
img[:,:,0]

array([[125, 125, 133, 128, 120, 116, 123],
       [125, 125, 133, 128, 120, 116, 123],
       [125, 125, 133, 128, 120, 116, 123],
       [125, 125, 133, 128, 120, 116, 123],
       [125, 125, 133, 128, 120, 116, 123],
       [123, 123, 113, 111, 120, 115, 120],
       [119, 119, 116, 115, 125, 112, 116]], dtype=uint8)

In [85]:
res

array([[125, 125, 125, 130, 132, 129, 125, 121, 118, 116, 119, 123],
       [125, 125, 125, 130, 132, 129, 125, 121, 118, 116, 119, 123],
       [125, 125, 125, 130, 132, 129, 125, 121, 118, 116, 119, 123],
       [125, 125, 125, 130, 132, 129, 125, 121, 118, 116, 119, 123],
       [125, 125, 125, 130, 132, 129, 125, 121, 118, 116, 119, 123],
       [125, 125, 125, 130, 132, 129, 125, 121, 118, 116, 119, 123],
       [125, 125, 125, 130, 132, 129, 125, 121, 118, 116, 119, 123],
       [125, 125, 125, 130, 132, 129, 125, 121, 118, 116, 119, 123],
       [124, 124, 124, 125, 125, 122, 121, 120, 118, 116, 118, 121],
       [123, 123, 122, 117, 114, 113, 114, 118, 118, 115, 117, 120],
       [121, 121, 120, 116, 114, 113, 115, 120, 119, 114, 115, 118],
       [119, 119, 118, 117, 115, 115, 117, 123, 120, 113, 113, 116]],
      dtype=uint8)

In [86]:
res_cv = cv2.resize(img, (12, 12), interpolation = cv2.INTER_LINEAR); res_cv[:,:,0]

array([[125, 125, 125, 129, 132, 129, 125, 121, 118, 116, 120, 123],
       [125, 125, 125, 129, 132, 129, 126, 121, 118, 116, 120, 123],
       [125, 125, 125, 129, 132, 129, 125, 121, 118, 116, 120, 123],
       [125, 125, 125, 129, 132, 129, 126, 121, 118, 116, 120, 123],
       [125, 125, 125, 129, 132, 129, 125, 121, 118, 116, 120, 123],
       [125, 125, 125, 129, 132, 129, 126, 121, 118, 116, 120, 123],
       [125, 125, 125, 129, 132, 129, 126, 121, 118, 116, 120, 123],
       [125, 125, 125, 129, 132, 129, 125, 121, 118, 116, 120, 123],
       [124, 124, 124, 124, 123, 121, 120, 120, 118, 116, 119, 122],
       [123, 123, 123, 117, 113, 112, 114, 119, 118, 115, 118, 120],
       [120, 120, 120, 117, 115, 114, 116, 122, 118, 113, 116, 118],
       [119, 119, 119, 117, 116, 115, 118, 124, 119, 112, 114, 116]],
      dtype=uint8)

## Bilinear interpolation as convolution

In [163]:

def get_upsampling_conv_properties(N, R):
    '''
    Returns parameters for upsampling convolution:
        # input, N X N --> output, R X R
        # 1. S := (N - 1) /  gcd(N-1, R-1)
        # 2. k := (R - 1) /  gcd(N-1, R-1)
        # 3. Convolution(kxk, stride=S, lhs_dilation=k, padding=k-1)
    '''
    def gcd(a,b):
        if(b==0):
            return a
        else:
            return gcd(b,a%b)
    
    _gcd_N_R = gcd(N, R)
    S = (N - 1) // _gcd_N_R
    K = (R - 1) // _gcd_N_R

    kernel_sz = K * 2 - 1
    padding = K-1
    lhs_dillation = K
    return K, kernel_sz, padding, lhs_dillation

def insert_zeros_2d(a, N=1, zero_point=0):
    '''
    Expand input 2d array for specified dillation
    '''
    out = np.ones( (N+1)*np.array(a.shape)-N, dtype=a.dtype) * zero_point
    out[::N+1,::N+1] = a
    return out

def get_bilinear_1D_kernel(n:int):
    '''
    Builds bilinear 1d kernel
    '''
    kernel_sz = n * 2 - 1
    kernel = np.zeros(shape=(kernel_sz,))
    for idx in range(0, n):
        v = idx + 1.0
        kernel[idx] = v
        kernel[n * 2 - 2 - idx] = v
    return kernel

def get_bilinear_2D_kernel(n:int):
    kernel_1d = get_bilinear_1D_kernel(n)
    kernel_2d = np.zeros((kernel_1d.shape[0], kernel_1d.shape[0]), dtype=np.int)
    kernel_2d[:,:] = kernel_1d
    kernel_2d = kernel_2d * kernel_2d.T
    return kernel_2d

In [164]:
get_bilinear_1D_kernel(n = 3)

array([1., 2., 3., 2., 1.])

In [165]:
arr = np.zeros((5,5))
arr[:,:] = get_bilinear_1D_kernel(n = 3)

In [166]:
arr = np.array([0, 3, 6, 9, 12, 15, 18, 21, 24]).reshape(3,3)
expanded = insert_zeros_2d(a=arr, N=2, zero_point=0)
expanded = np.pad(expanded, pad_width=2)
expanded

array([[ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  3,  0,  0,  6,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  9,  0,  0, 12,  0,  0, 15,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0, 18,  0,  0, 21,  0,  0, 24,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0]])

In [131]:
kernel = np.array([1,2,3,2,1,2,4,6,4,2,3,6,9,6,3,2,4,6,4,2,1,2,3,2,1]).reshape(5, 5)

In [137]:
do_bilinear_resize(img=arr[..., np.newaxis], new_size=(4,4)).reshape(4,4)

array([[ 0,  2,  4,  6],
       [ 6,  8, 10, 12],
       [12, 14, 16, 18],
       [18, 20, 22, 24]], dtype=uint8)

In [144]:
from numpy.lib.stride_tricks import as_strided

def conv2d(a, b, s=1):
    Hout = (a.shape[1] - b.shape[0]) // s + 1
    Wout = (a.shape[2] - b.shape[1]) // s + 1
    Stride = (a.strides[0], a.strides[1] * s, a.strides[2] * s, a.strides[1], a.strides[2], a.strides[3])

    a = as_strided(a, (a.shape[0], Hout, Wout, b.shape[0], b.shape[1], a.shape[3]), Stride)

    return np.tensordot(a, b, axes=3)

# test
conv2d(a=expanded.reshape(1,11,11,1), b=kernel.reshape(5,5,1), s=2) / 9

array([[[ 0.,  2.,  4.,  6.],
        [ 6.,  8., 10., 12.],
        [12., 14., 16., 18.],
        [18., 20., 22., 24.]]])

In [171]:
kernel_1d = get_bilinear_1D_kernel(n = 3); kernel_1d.reshape(1,1,5,1).T.shape

(1, 5, 1, 1)

In [187]:
arr1 = conv2d(a=expanded.reshape(1,11,11,1), b=kernel_1d.reshape(1, 5, 1), s=2)
arr2 = conv2d(a=arr1[..., np.newaxis], b=kernel_1d.reshape(1, 1, 5).T, s=2)

In [188]:
arr2

array([[[54., 90.]]])

In [ ]:
# TODO: Decompose and see

+ OpenCV resize() result is wrong - https://stackoverflow.com/a/43701718/6561141
+ Resize an image with bilinear interpolation without imresize https://stackoverflow.com/a/26143655/6561141
+  Richard Alan Peters' II digital image processing slides on interpolation - https://ia802707.us.archive.org/23/items/Lectures_on_Image_Processing/EECE_4353_15_Resampling.pdf
+ Understanding Bilinear Image Resizing https://chao-ji.github.io/jekyll/update/2018/07/19/BilinearResize.html
+ bilinear interpolation as convolution - https://github.com/tensorflow/tensorflow/blob/cddf9415564b16c2bc234df68d3eb44fc8689dae/tensorflow/compiler/tf2xla/kernels/image_resize_ops.cc